# **Interactive Brokers Python API 설정하기**

_1장에서 9장까지_ 알고리즘 트레이딩의 기초와 기술에 대해 배웠습니다. 이제 Python을 사용하여 **Interactive Brokers(IB) API**와 **Trader Workstation(TWS)**를 통해 이를 실제로 적용해 보겠습니다. TWS는 전문 및 소매 트레이더 모두가 사용하는 고급 거래 플랫폼입니다. 다른 대안으로는 Alpaca, Think Or Swim, Tasty Trade 및 Tradier가 있습니다. 계좌를 개설할 때는 시장 접근성, 계좌 유형, 수수료 및 기타 요소를 고려해야 합니다. 브로커마다 비용 및 제한사항이 다를 수 있습니다.

TWS는 다양한 거래 도구와 기능으로 유명하여 거래 경험을 크게 향상시킵니다. TWS에는 강력한 리스크 관리 도구가 갖춰져 있어 트레이더가 잠재적인 거래 리스크를 효과적으로 관리 및 완화할 수 있습니다. IB는 전 세계 33개국의 135개 시장에 걸쳐 다양한 금융 상품을 거래할 수 있는 탁월한 글로벌 시장 접근성을 제공합니다. 이러한 기능은 포트폴리오를 글로벌하게 다변화하고자 하는 트레이더에게 유용합니다.

또한, TWS의 모의 거래 기능은 초보자 및 숙련된 트레이더 모두에게 실시간 시장 상황을 사용하여 거래 전략을 테스트하고 개선할 수 있는 위험이 없는 환경을 제공합니다. 이 기능은 알고리즘 트레이더가 실제 돈을 위험에 노출하지 않고 거래 전략을 개발 및 테스트할 수 있도록 돕는 데 매우 유용합니다. 마지막으로, TWS의 API 통합은 모든 TWS 기능을 Python을 통해 접근할 수 있는 API를 제공한다는 점에서 큰 이점입니다. IB API는 알고리즘 트레이더가 거래 전략을 자동화할 수 있도록 특히 유용합니다. 앞으로의 세 개 장에서 모의 거래 계정과 IB API를 활용할 것입니다.

다음 장에서는 다음과 같은 내용을 다룹니다:

- 알고리즘 트레이딩 앱 구축하기
- IB API를 사용하여 `Contract` 객체 생성하기
- IB API를 사용하여 `Order` 객체 생성하기
- 과거 시장 데이터 가져오기
- 실시간 틱 데이터 스트리밍하기
- 실시간 틱 데이터를 로컬 SQL 데이터베이스에 저장하기

---

## **알고리즘 트레이딩 앱 구축하기**

IB API를 사용할 때는 다양한 거래 앱에 걸쳐 재사용할 수 있는 코드가 많이 있습니다. TWS에 연결하고 주문을 생성하며 데이터를 다운로드하는 방법은 거래 전략에 관계없이 동일하기 때문에, 재사용 가능한 코드를 먼저 작성하는 것이 모범 사례입니다. 하지만 알고리즘 트레이딩 앱을 구축하기 전에 TWS와 IB API를 먼저 설치해야 합니다.

IB API를 사용할 때 가장 중요한 세 가지 개념은 다음과 같습니다:

1. **IB API의 아키텍처**: 비동기 모델로 작동하며, 작업이 순차적이고 차단되는 방식이 아니라 요청을 통해 실행됩니다. 요청에 대한 응답은 콜백(callback)을 통해 처리됩니다.

2. **상속(Inheritance)**: 모든 프로그래밍 언어에 공통적으로 사용되는 개념으로, 자식 클래스가 부모 클래스의 속성과 메서드를 획득합니다. 우리의 트레이딩 앱은 IB API에서 제공하는 두 개의 부모 클래스(`Client`와 `EWrapper`)의 기능을 상속받게 됩니다.

3. **오버라이딩(Overriding)**: 자식 클래스가 부모 클래스에서 정의된 메서드의 구체적인 구현을 제공하는 개념입니다.

이 세 가지 개념이 우리의 알고리즘 트레이딩 앱의 재사용 가능한 부분에 어떻게 적용되는지 더 자세히 배울 것입니다.

---

### **준비하기...**

IB API를 설치하기 전에 로컬 컴퓨터에 TWS를 먼저 설치해야 합니다. TWS는 IB 웹사이트에서 다운로드할 수 있습니다:  
[Interactive Brokers TWS 다운로드](https://www.interactivebrokers.com/en/trading/tws.php#tws-software)  
Windows, Mac 및 Linux용으로 제공됩니다.

설치가 완료되면 TWS를 시작할 수 있습니다.

계정이 있는 경우 로그인하세요. 계정이 없으면 로그인 화면의 **데모 계정(Demo Account)** 옵션을 사용할 수 있습니다.

다음으로 Python 앱이 TWS에 연결할 수 있도록 몇 가지 설정을 변경해야 합니다. **Trader Workstation Configuration**으로 이동한 다음 다음 경로를 따르세요:  
**Edit → Global Configuration → API → Settings**.  

다음과 같은 화면이 표시되어야 합니다:

<img src="./images/fig_10_01.png" width=800>

Figure 10.1: Global configuration settings

다음 내용을 원본 형식 그대로 한국어로 번역합니다.


**반드시** **ActiveX 및 소켓 클라이언트 사용(Enable ActiveX and Socket Clients)** 을 확인하세요. 추가 보호를 위해 IB에 주문을 보내지 않도록 하려면 **읽기 전용 API(Read-Only API)** 를 확인하세요. 마지막으로 보안을 위해 **로컬호스트에서만 연결 허용(Allow connections from localhost only)** 을 선택하세요.

**소켓 포트(Socket port)** 를 메모하세요. 이는 Python을 통해 연결할 때 필요합니다. 라이브 계정 또는 모의 거래 계정으로 로그인했는지에 따라 포트는 **7497** 또는 **7496**입니다.

> **중요한 참고 사항(IMPORTANT NOTE)**  
> IB API를 실시간으로 사용하려면 IBKR Pro 계정이 필요합니다(IBKR Lite가 아님). IB 계정을 설정하기 원하지 않거나 설정할 수 없는 경우, 무료 데모 계정을 사용할 수 있습니다. 하지만 무료 데모 계정에서는 시장 데이터를 다운로드하거나 스트리밍할 수 없습니다.


TWS의 오른쪽 상단 모서리 근처에서 **DATA**라는 초록색 버튼을 볼 수 있습니다. 이를 클릭하면 연결된 데이터 팜을 확인할 수 있습니다. 아래쪽 근처에 제목이 **API Connections (listening on *:7497)** 라고 표시된 빈 섹션을 확인하세요. 이 섹션이 표시되면 API 연결이 올바르게 활성화되었음을 의미합니다.

<img src="./images/fig_10_02.png" width=800>

Figure 10.2: Data connections screen showing empty API connections

TWS 설정이 완료되면 IB API를 설치할 수 있습니다. Interactive Brokers GitHub 페이지( http://interactivebrokers.github.io/ )에서 Python API를 다운로드하세요. 다운로드 페이지의 지침을 따라 운영체제에 맞는 최신 안정 버전의 API를 다운로드하고 설치하세요.

IB API가 설정되면 **`trading-app`**이라는 새 디렉토리를 생성하세요. 그 안에 다음 Python 스크립트 파일들을 만드세요:

- `__init__.py`  
- `app.py`  
- `client.py`  
- `wrapper.py`  
- `contract.py`  
- `order.py`  
- `utils.py`  

---

### **어떻게 하는지...**

우리는 IB API를 통해 TWS에 연결하는 데 필요한 코드를 설정하는 것으로 시작하겠습니다.

1. **`client.py`** 파일에 다음 코드를 추가하세요. 이 코드는 IB API의 기본 클래스를 가져와 거래 애플리케이션을 구축하는 데 사용할 사용자 정의 클래스를 구현합니다:  

```python
from ibapi.client import EClient

class IBClient(EClient):
    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)
```

2. **`wrapper.py`** 파일에 다음 코드를 추가하세요. 이 코드도 IB API의 기본 클래스를 가져와 거래 애플리케이션을 구축하는 데 사용할 사용자 정의 클래스를 구현합니다:  

```python
from ibapi.wrapper import EWrapper

class IBWrapper(EWrapper):
    def __init__(self):
        EWrapper.__init__(self)
```

3. **`app.py`** 파일에 다음 코드를 추가하세요. 이 코드는 거래 애플리케이션으로 발전시킬 사용자 정의 클래스를 구현합니다:  

```python
import threading
import time
from wrapper import IBWrapper
from client import IBClient

class IBApp(IBWrapper, IBClient):
    def __init__(self, ip, port, client_id):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)
        self.connect(ip, port, client_id)
        
        thread = threading.Thread(target=self.run, daemon=True)
        thread.start()
        time.sleep(2)

if __name__ == "__main__":
    app = IBApp("127.0.0.1", 7497, client_id=10)
    time.sleep(30)
    app.disconnect()
```

---

### **작동 방식(How it works...)**

IB API의 아키텍처는 비동기 모델(asynchronous model)로 작동하며, 이는 상호 작용 및 데이터 흐름 메커니즘을 이해하는 데 필수적입니다. 이 모델에서는 작업이 선형으로 실행되지 않습니다. 대신 요청(request)을 통해 작업이 시작되며, 해당 요청에 대한 응답(response)은 콜백(callback)을 통해 처리됩니다. 이로 인해 시스템은 매우 효율적이고 반응성이 뛰어납니다.

**요청-콜백 패턴(Request-callback pattern)**

IB API와 상호 작용할 때 일반적으로 작업을 요청하는 것으로 시작합니다. 예를 들어, 시장 데이터를 가져오거나 주문을 실행하는 요청입니다. 이 요청은 IB 서버와의 통신을 시작하는 API 구성 요소인 **`EClient`**의 인스턴스를 통해 이루어집니다. **`EClient`**는 요청을 보낸 후 응답을 기다리지 않고 다른 작업을 계속 수행합니다. 이와 같은 비차단(non-blocking) 패턴은 Python의 **Global Interpreter Lock(GIL)** 문제를 피하고 아키텍처의 비동기성을 보여줍니다.

이러한 요청에 대한 응답은 **`EClient`**가 직접 수신하지 않습니다. 대신 IB 서버로부터 응답이나 데이터를 수신하면 **`EWrapper`**라는 구성 요소가 이를 처리합니다. **`EWrapper`**는 기본적으로 응답이나 데이터가 도착할 때 호출되는 일련의 콜백 함수(callback functions)입니다. 예를 들어, 시장 데이터를 요청할 경우 해당 데이터가 도착하면 **`EWrapper`**가 특정 메소드를 호출합니다. 사용자는 이러한 메소드를 **`EWrapper`**에서 구현하여 데이터나 응답을 처리하는 방법을 정의합니다.

이러한 역할 분리는 **`EClient`**가 요청을 관리하고 **`EWrapper`**가 응답을 처리하도록 함으로써 코드 구조가 더 체계적이고 관리하기 쉬워집니다. 또한 다중 데이터 스트림과 요청을 동시에 효율적으로 처리할 수 있게 합니다. 비동기 모델 덕분에 애플리케이션은 데이터나 이벤트가 발생할 때 이를 즉시 처리하여 시장 변화에 대한 신속한 대응이 중요한 실시간 거래 애플리케이션에 매우 적합합니다.

**상속과 오버라이딩(Inheritance and overriding)**

**`EWrapper`**는 일련의 콜백 메소드를 정의하는 인터페이스 클래스입니다. 이 메소드들은 **`EWrapper`**를 상속받은 사용자 정의 클래스에 의해 오버라이드(override)될 목적으로 설계되었습니다. 이러한 상속을 통해 시장 데이터 수신이나 주문 업데이트와 같은 다양한 이벤트에 대한 응답을 사용자 정의할 수 있습니다. 마찬가지로 **`EClient`**도 상속받거나 메소드를 오버라이드하여 IB 서버에 요청을 보내는 방식을 사용자 정의할 수 있습니다. 이를 통해 거래 시스템과의 상호작용에 대한 유연성과 제어권을 확보할 수 있습니다.

**우리의 거래 앱 클래스(Our trading app class)**

메인 거래 앱은 사용자 정의 **`IBWrapper`** 및 **`IBClient`** 클래스를 상속받는 커스텀 클래스입니다. 이 클래스는 IB API에 대한 연결을 초기화하고 관리하도록 설계되었습니다. (참고: **`IBWrapper`**와 **`IBClient`**는 각각 IB API의 **`EWrapper`** 및 **`EClient`** 클래스를 상속합니다.)

**`IBApp`** 클래스의 **`__init__`** 메소드에서는 **`IBWrapper.__init__(self)`** 및 **`IBClient.__init__(self, wrapper=self)`**를 사용하여 상위 클래스에서 상속받은 환경과 속성을 초기화합니다. 특히 **`IBClient`** 초기화는 **`IBWrapper`** 인스턴스의 참조를 필요로 하며, 이를 위해 **`self`**가 전달됩니다.

다음으로 **`connect`** 메소드를 **`ip`**, **`port`**, **`client_id`** 매개변수와 함께 호출합니다. 이 메소드는 **`IBClient`**로부터 상속되며 TWS에 대한 연결을 설정하는 데 사용됩니다. 연결이 완료되면 Python의 **`threading`** 모듈을 사용하여 새 스레드(thread)를 생성하고 시작합니다. 이 스레드는 **`run`** 메소드를 데몬 모드(daemon mode)로 실행합니다. 데몬 모드에서는 이 스레드가 백그라운드에서 실행되며, 메인 프로그램이 종료되면 자동으로 종료됩니다.  

**`run`** 메소드는 IB API로부터 들어오는 메시지와 이벤트를 비동기적으로 처리합니다. 이를 별도의 스레드에서 실행함으로써 **`IBApp`**이 API 메시지를 메인 실행 흐름을 차단하지 않고 처리할 수 있습니다.

---

### **추가 설명(There’s more...)**

이제 기본 클래스가 설정되었으므로 연결을 테스트할 수 있습니다.

터미널에서 다음 명령어를 실행하세요:  

```bash
python app.py
```

화면에 일련의 오류 메시지가 표시될 수 있습니다. 하지만 걱정하지 않아도 됩니다. 실제 오류가 아니라 IB 데이터 팜에 성공적으로 연결되었음을 알리는 메시지입니다.

<img src="./images/fig_10_03.png" width=800>

Figure 10.3: Output from the IB API showing we successfully connected 

>Important note 
>
>Error code -1 is not actually an error. These are messages indicating successful connections to the IB data farms. 

Also note in the TWS data ​connection screen, we now have a peer IP connected to TWS. That’s the Python trading app we built!

<img src="./images/fig_10_04.png" width=800>

Figure 10.4: TWS data connection screen showing our Python trading app connected to TWS

---

### See also 

The IB API has extensive ​documentation: 
- Documentation for the initial setup: https://interactivebrokers.github.io/tws-api/initial_setup.html
- Documentation about the EClient and EWrapper classes: https://interactivebrokers.github.io/tws-api/client_wrapper.html
- Documentation about connectivity: https://interactivebrokers.github.io/tws-api/connection.html

---

## **IB API로 계약 객체 생성하기**

시장 데이터를 요청하거나 주문을 생성할 때는 IB의 **`Contract`** 객체를 사용합니다. IB의 **`Contract`** 객체에는 해당 금융 상품을 올바르게 식별하는 데 필요한 모든 정보가 포함됩니다. 하나의 클래스만 사용하여 주식, 옵션, 선물 등 다양한 금융 상품을 표현할 수 있습니다. 이 단계에서는 IB **`Contract`**를 생성합니다.

**`Contract`** 클래스는 거래하거나 조회하려는 금융 상품의 사양(specifications)을 정의하는 데 사용됩니다. 이 클래스는 주식, 옵션, 선물, 외환, 채권 등 다양한 자산 클래스에서 금융 상품을 고유하게 식별하는 데 필요한 세부 정보를 모두 포함합니다. 그러나 많은 경우, 이 ID를 직접 지정할 필요는 없습니다. 대신, IB 시스템이 계약을 고유하게 식별하는 데 사용하는 다음과 같은 설명적 속성(descriptive attributes)을 제공하는 것이 일반적입니다.

- **`symbol`**: 자산의 티커 심볼(ticker symbol)  
- **`secType`**: 금융 상품 유형 지정(예: 주식은 `STK`, 옵션은 `OPT`, 선물은 `FUT`)  
- **`expiry`**: 파생상품(derivative instruments)의 만기일(expiration date)  
- **`strike`**: 옵션의 행사가격(strike price)  
- **`right`**: 옵션 유형(call 또는 put) 표시  
- **`multiplier`**: 레버리지(leverage) 또는 계약 단위(contract size) 정의  
- **`exchange`**: 해당 자산이 거래되는 주요 거래소(primary exchange)  

더 복잡한 금융 상품의 경우 다음과 같은 추가 속성을 지정할 수 있습니다.  
- **`currency`**: 여러 통화 또는 국제 시장에서 거래되는 자산의 통화(currency)  
- **`lastTradeDateOrContractMonth`**: 선물 및 옵션의 계약 월(contract month)을 지정하기 위해 사용  
- **`includeExpired`**: 옵션과 선물의 경우, 만료된 계약을 고려할지 여부를 지정  
- **`localSymbol`** 및 **`primaryExchange`** 속성: 표준 심볼이 고유하지 않을 수 있는 금융 상품에 대해 더 구체적인 계약 세부 정보를 제공  

이번 단계에서는 선물, 주식 및 옵션 계약을 위한 사용자 정의 함수(custom functions)를 생성할 것입니다.

> **💡 IMPORTANT NOTE (중요한 참고사항)**  
>
> 모든 금융 상품은 IB 시스템에서 계약(contract)으로 간주됩니다. 이는 주식이나 외환(FX)을 고려할 때 다소 놀라울 수 있습니다.

---

### **준비하기...**


`trading-app` 디렉토리에 **`contract.py`** 파일을 생성했다고 가정합니다. 아직 생성하지 않았다면 지금 생성하세요.

---

### **어떻게 하는지...**

우리는 다양한 종류의 계약 인스턴스를 생성하고, 해당 금융 상품 유형에 맞게 속성을 설정한 후 반환하는 맞춤형 함수를 만들 것입니다.

다음 코드를 `contract.py` 파일에 추가하세요.

```python
from ibapi.contract import Contract

def future(symbol, exchange, contract_month):
    contract = Contract()
    contract.symbol = symbol
    contract.exchange = exchange
    contract.lastTradeDateOrContractMonth = contract_month
    contract.secType = "FUT"
    return contract

def stock(symbol, exchange, currency):
    contract = Contract()
    contract.symbol = symbol
    contract.exchange = exchange
    contract.currency = currency
    contract.secType = "STK"
    return contract

def option(symbol, exchange, contract_month, strike, right):
    contract = Contract()
    contract.symbol = symbol
    contract.exchange = exchange
    contract.lastTradeDateOrContractMonth = contract_month
    contract.strike = strike
    contract.right = right
    contract.secType = "OPT"
    return contract
```

---

### **작동 방식...**

이 함수들은 계약을 생성할 때 일반적으로 반복되는 패턴을 캡슐화하도록 설계되었습니다.  
- `future` 함수는 선물 계약을 나타내는 `Contract` 객체를 생성합니다. 이 함수는 계약을 설정하기 위해 `symbol`, `exchange`, `contract_month`의 세 가지 매개변수를 사용합니다.  
  함수를 호출하면 IB API의 핵심 구성 요소인 `Contract` 클래스의 인스턴스를 생성합니다. 해당 인스턴스는 변수 `contract`에 저장되며, 선물 계약의 `symbol`, 거래소 `exchange`, 그리고 만기일을 나타내는 `lastTradeDateOrContractMonth`와 같은 속성들로 구성됩니다. 또한, `secType`은 해당 계약이 선물 계약임을 명시하기 위해 "FUT"로 설정됩니다.

- `stock` 및 `option` 함수도 동일한 패턴을 따릅니다.

---

### **더 있습니다...**

이제 계약 함수들을 정의했으니 이를 사용해 보겠습니다. `app.py`를 열고, `contract` 임포트 바로 아래에 새 임포트를 추가하세요.

```python
from contract import stock, future, option
```

그런 다음 앱 인스턴스화 바로 아래에 다음 코드를 추가하세요.

```python
aapl = stock("AAPL", "SMART", "USD")
gbl = future("GBL", "EUREX", "202403")
pltr = option("PLTR", "BOX", "20240315", 20, "C")
```

이 변경 사항의 결과는 `app.py` 파일의 다음 코드입니다.

```python
import threading
import time
from wrapper import IBWrapper
from client import IBClient
from contract import stock, future, option

class IBApp(IBWrapper, IBClient):
    def __init__(self, ip, port, client_id):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)
        self.connect(ip, port, client_id)
        thread = threading.Thread(target=self.run, daemon=True)
        thread.start()
        time.sleep(2)

if __name__ == "__main__":
    app = IBApp("127.0.0.1", 7497, client_id=10)
    aapl = stock("AAPL", "SMART", "USD")
    gbl = future("GBL", "EUREX", "202403")
    pltr = option("PLTR", "BOX", "20240315", 20, "C")
    time.sleep(30)
    app.disconnect()
```

이 코드를 실행하면 주식, 선물 및 옵션 계약을 생성하고, 30초 동안 대기한 후 연결을 해제합니다.

---

### **참고 사항**  

IB API에서는 모든 금융 상품이 계약으로 간주됩니다. 여기에는 주식, ETF, 채권, 옵션 및 선물이 포함됩니다. 계약 객체에 대해 더 자세히 알고 싶다면 다음 URL을 참조하세요: [https://interactivebrokers.github.io/tws-api/contracts.html](https://interactivebrokers.github.io/tws-api/contracts.html)

---

## **IB API를 사용하여 Order 객체 생성하기**

`Contract` 객체가 IB가 금융 상품에 필요한 모든 정보를 캡슐화하는 것과 유사하게, `Order` 객체는 IB가 시장에서 주문을 정확하게 실행하는 데 필요한 모든 정보를 포함합니다. IB는 단순한 시장 주문부터 고급 알고리즘까지 수십 가지 주문 유형으로 유명합니다. `Order` 클래스의 주요 속성은 다음과 같습니다.

- **`orderId`**: 클라이언트가 일반적으로 할당하는 고유 식별자
- **`action`**: 주문의 작업 유형을 지정하며, 예를 들어 `BUY` 또는 `SELL`
- **`totalQuantity`**: 매수 또는 매도할 자산의 수량
- **`orderType`**: 주문의 유형을 정의하며, 예: `LMT`(지정가 주문) 또는 `MKT`(시장가 주문)
- **`lmtPrice`**: 지정가 주문의 제한 가격
- **`auxPrice`**: 스톱 또는 스톱-리밋 주문의 스톱 가격 지정 시 사용
- **`tif`**: Time in force, 주문이 활성 상태로 유지되는 기간을 결정함
- **`outsideRTH`**: 정규 거래 시간이 아닌 시간에 주문을 실행할 수 있는지 여부를 나타내는 부울 값
- **`account`**: 주문이 실행될 계정을 지정

추가 속성을 설정하여 주문을 더 세부적으로 사용자 지정할 수 있습니다. 이러한 속성에는 스톱 주문의 `stopPrice`, 트레일링 스톱 주문의 `trailStopPrice`, 다양한 조건부 주문 속성이 포함됩니다. `Order` 클래스는 수수료, 마진 및 기타 거래 관련 매개변수를 지정하기 위한 속성도 지원합니다.

---

### **준비하기...**

`trading-app` 디렉토리에 `order.py` 파일을 생성했다고 가정합니다. 그렇지 않다면 지금 생성하세요.

---

### **어떻게 하는지...**

우리는 다양한 유형의 계약 인스턴스를 생성하고, 해당 금융 상품 유형과 관련된 속성을 설정한 후 이를 반환하는 맞춤형 함수를 만들 것입니다.

다음 코드를 `order.py` 파일에 추가하세요. 이 코드는 IB API를 사용하여 `Order` 객체를 생성하고 반환하는 맞춤형 함수를 구현합니다.

```python
from ibapi.order import Order

BUY = "BUY"
SELL = "SELL"

def market(action, quantity):
    order = Order()
    order.action = action
    order.orderType = "MKT"
    order.totalQuantity = quantity
    return order

def limit(action, quantity, limit_price):
    order = Order()
    order.action = action
    order.orderType = "LMT"
    order.totalQuantity = quantity
    order.lmtPrice = limit_price
    return order

def stop(action, quantity, stop_price):
    order = Order()
    order.action = action
    order.orderType = "STP"
    order.auxPrice = stop_price
    order.totalQuantity = quantity
    return order
```

---

### **작동 방식...**

시장가 주문은 현재 시장 가격으로 즉시 실행됩니다. 이는 정확한 거래 가격보다 실행 속도를 우선시하므로 빠르게 체결되지만 반드시 특정 가격 지점에서 이루어지지는 않습니다. 시장가 주문은 실행의 확실성이 가격보다 중요한 상황에서 일반적으로 사용됩니다. IB API를 사용하여 시장가 주문을 정의하려면 먼저 `Order` 객체를 인스턴스화한 다음, `action` 속성에 `BUY` 또는 `SELL`을 지정하고, `totalQuantity` 속성에 원하는 수량을 할당하며, `orderType` 속성에 `MKT`를 설정합니다.

이와 유사하게, `orderType`을 `LMT`로 지정하고 `lmtPrice`에 제한 가격을 설정하여 지정가 주문을 생성할 수 있습니다. 지정가 주문은 트레이더가 매수 시 지불할 수 있는 최대 가격 또는 매도 시 수용할 수 있는 최소 가격을 지정할 수 있게 해줍니다. 시장가 주문과 달리, 지정가 주문은 시장 가격이 제한 가격에 도달하거나 더 나은 가격일 경우에만 체결되므로 가격 통제는 가능하지만 실행이 보장되지는 않습니다.

손절매 주문은 지정된 스톱 가격에 도달하면 자동으로 주문을 실행하여 손실을 제한합니다. 손절매 주문을 생성하려면 `orderType`을 `STP`로 설정하고 `auxPrice`를 원하는 스톱 가격으로 지정합니다.

---

### **더 있습니다...**

이제 주문 함수들을 정의했으니 이를 사용해 보겠습니다. `app.py`를 열고, `contract` 임포트 바로 아래에 새 임포트를 추가하세요.

```python
from order import limit, BUY
```

그런 다음 계약 생성 직후에 다음 코드를 추가하세요.

```python
limit_order = limit(BUY, 100, 190.00)
```

이 변경 사항의 결과는 `app.py` 파일의 다음 코드입니다.

```python
import threading
import time
from wrapper import IBWrapper
from client import IBClient
from contract import stock, future, option
from order import limit, BUY

class IBApp(IBWrapper, IBClient):
    def __init__(self, ip, port, client_id):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)
        self.connect(ip, port, client_id)
        thread = threading.Thread(target=self.run, daemon=True)
        thread.start()
        setattr(self, "thread", thread)

if __name__ == "__main__":
    app = IBApp("127.0.0.1", 7497, client_id=10)
    aapl = stock("AAPL", "SMART", "USD")
    gbl = future("GBL", "EUREX", "202403")
    pltr = option("PLTR", "BOX", "20240315", 20, "C")
    limit_order = limit(BUY, 100, 190.00)
    time.sleep(30)
    app.disconnect()
```

이 코드를 실행하면 지정가 주문 객체를 생성하고, 30초 동안 대기한 후 연결을 해제합니다. IB API를 사용하여 주문을 제출하는 방법은 **Chapter 11, Manage Orders, Positions, and Portfolios with the IB API**에서 다룰 예정입니다.

---

### **참고 사항**  

`Order` 객체와 사용 가능한 주문 유형에 대해 더 자세히 알고 싶다면 다음 문서를 참조하세요:  
[https://interactivebrokers.github.io/tws-api/available_orders.html](https://interactivebrokers.github.io/tws-api/available_orders.html)  

---

## **과거 시장 데이터 가져오기**

IB API를 사용하여 과거 시장 데이터를 요청하는 것은 비동기 방식의 이벤트 기반 데이터 검색 프로세스입니다. 이 프로세스는 `reqHistoricalData` 메소드를 호출하여 시작되며, 해당 금융 상품의 식별자, 데이터를 요청할 기간, 바(bar) 크기 및 필요한 데이터 유형과 같은 매개변수를 지정합니다. 요청이 이루어지면 IB API는 이를 처리하고 데이터를 전송하기 시작합니다. 하지만 모든 데이터를 받을 때까지 대기하지 않고, API는 콜백 메커니즘을 사용합니다. 특히, `historicalData` 메소드는 IB로부터 수신된 각 데이터 조각마다 비동기적으로 호출됩니다. 이 메소드의 각 호출은 특정 시간 간격의 시장 데이터 스냅샷을 제공하며, 이를 처리하거나 저장할 수 있습니다. 이 레시피에서는 코드를 설정하여 과거 시장 데이터를 요청하고 수신할 수 있게 하겠습니다.

---

### **준비하기...**

`trading-app` 디렉토리에 `app.py`, `client.py`, 및 `wrapper.py` 파일이 생성되었다고 가정합니다. 그렇지 않다면 지금 생성하세요.

---

### **어떻게 하는지...**

우리는 `app.py`, `client.py`, 및 `wrapper.py` 파일을 업데이트하여 과거 시장 데이터를 요청할 것입니다.

1. **`client.py` 파일을 열고, 파일 맨 위에 다음 임포트를 추가하세요.**

```python
import time
import pandas as pd
```

2. **문자열 목록을 상수로 만들어 주세요. 이 목록의 문자열은 DataFrame의 열(column)을 나타내며, 여기에 과거 시장 데이터를 채울 것입니다.**

```python
TRADE_BAR_PROPERTIES = ["time", "open", "high", "low", "close", "volume"]
```

3. **`client.py` 파일의 `IBClient` 클래스 내부에 다음 메소드를 추가하세요.**

```python
def get_historical_data(self, request_id, contract, duration, bar_size):
    self.reqHistoricalData(
        reqId=request_id,
        contract=contract,
        endDateTime="",
        durationStr=duration,
        barSizeSetting=bar_size,
        whatToShow="MIDPOINT",
        useRTH=1,
        formatDate=1,
        keepUpToDate=False,
        chartOptions=[],
    )
    time.sleep(5)

    bar_sizes = ["day", "D", "week", "W", "month"]
    if any(x in bar_size for x in bar_sizes):
        fmt = "%Y%m%d"

    else:
        fmt = "%Y%m%d %H:%M:%S %Z"

    data = self.historical_data[request_id]
    df = pd.DataFrame(data, columns=TRADE_BAR_PROPERTIES)
    df.set_index(pd.to_datetime(df.time, format=fmt), inplace=True)
    df.drop("time", axis=1, inplace=True)
    df["symbol"] = contract.symbol
    df.request_id = request_id
    return df
```

4. **여러 계약에 대한 데이터를 요청할 수 있는 메소드를 생성하세요.**

```python
def get_historical_data_for_many(self, request_id, contracts, duration, bar_size, col_to_use="close"):
    dfs = []
    for contract in contracts:
        data = self.get_historical_data(
            request_id, contract, duration, bar_size
        )
        dfs.append(data)
        request_id += 1
    return (
        pd.concat(dfs)
        .reset_index()
        .pivot(
            index="time",
            columns="symbol",
            values=col_to_use
        )
    )
```

변경사항을 적용한 후 `client.py` 파일의 코드는 다음과 같습니다.

```python
import time
import pandas as pd
from ibapi.client import EClient

TRADE_BAR_PROPERTIES = ["time", "open", "high", "low", "close", "volume"]

class IBClient(EClient):
    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)

    def get_historical_data(self, request_id, contract, duration, bar_size):
        self.reqHistoricalData(
            reqId=request_id,
            contract=contract,
            endDateTime="",
            durationStr=duration,
            barSizeSetting=bar_size,
            whatToShow="MIDPOINT",
            useRTH=1,
            formatDate=1,
            keepUpToDate=False,
            chartOptions=[],
        )
        time.sleep(5)

        bar_sizes = ["day", "D", "week", "W", "month"]
        if any(x in bar_size for x in bar_sizes):
            fmt = "%Y%m%d"
        else:
            fmt = "%Y%m%d %H:%M:%S %Z"

        data = self.historical_data[request_id]
        df = pd.DataFrame(
            data,
            columns=TRADE_BAR_PROPERTIES
        )
        df.set_index(pd.to_datetime(df.time, format=fmt), inplace=True)
        df.drop("time", axis=1, inplace=True)
        df["symbol"] = contract.symbol
        df.request_id = request_id
        return df

    def get_historical_data_for_many(self, request_id, contracts, duration, bar_size, col_to_use="close"):
        dfs = []
        for contract in contracts:
            data = self.get_historical_data(
                request_id, contract, duration, bar_size
            )
            dfs.append(data)
            request_id += 1
        return (
            pd.concat(dfs)
            .reset_index()
            .pivot(
                index="time",
                columns="symbol",
                values=col_to_use
            )
        )
```

5. **이제 `wrapper.py` 파일을 열고, `IBWrapper` 클래스의 `__init__` 메소드에 다음 코드를 추가하세요.**

```python
self.historical_data = {}
```

6. **그런 다음, `IBWrapper` 클래스에 다음 메소드를 추가하세요.**

```python
def historicalData(self, request_id, bar):
    bar_data = (
        bar.date,
        bar.open,
        bar.high,
        bar.low,
        bar.close,
        bar.volume,
    )

    if request_id not in self.historical_data.keys():
        self.historical_data[request_id] = []
    self.historical_data[request_id].append(bar_data)
```

변경사항을 적용한 후 `wrapper.py` 파일의 코드는 다음과 같습니다.

```python
from ibapi.wrapper import EWrapper

class IBWrapper(EWrapper):
    def __init__(self):
        EWrapper.__init__(self)
        self.historical_data = {}

    def historicalData(self, request_id, bar):
        bar_data = (
            bar.date,
            bar.open,
            bar.high,
            bar.low,
            bar.close,
            bar.volume,
        )

        if request_id not in self.historical_data.keys():
            self.historical_data[request_id] = []
        self.historical_data[request_id].append(bar_data)
```

---

### **작동 방식**

IB API를 통해 과거 데이터를 가져오는 것은 요청-콜백(request-callback) 아키텍처의 예시를 명확히 보여줍니다. 먼저 데이터를 요청한 다음 콜백을 통해 데이터를 처리합니다.

**과거 데이터 요청하기**

- `get_historical_data` 메소드는 네 가지 매개변수를 받습니다:  
  - **`request_id`**: 데이터 요청을 위한 고유 식별자입니다.  
  - **`contract`**: 요청할 금융 상품을 지정합니다.  
  - **`duration`**: 과거 데이터의 시간 범위를 나타냅니다.  
  - **`bar_size`**: 데이터의 세분화 수준을 정의합니다.

다음 단계에서는 유효한 `duration` 문자열을 설명합니다.

Figure 10.5: Valid duration strings for requesting historical data 

<img src="./images/fig_10_05.png" width=200>

The valid bar size strings are as follows:

<img src="./images/fig_10_06.png" width=800>

Figure 10.6: Valid bar sizes for requesting historical data

The method begins by calling reqHistoricalData, which is a method provided by the IB API to request historical data. The parameters passed to this function include request_id, the contract object, an empty string for endDateTime (indicating the request is for the most recent ​data up to the current date), the duration of historical data, bar_size, and other settings such as whatToShow (indicating the type of data required), useRTH (data within regular trading hours), formatDate (the format of the returned date), and keepUpToDate (indicating no need for live updates). These are hardcoded in our implementation, but you can modify them to allow the user to pass their own options. 

The available kinds of historic data type strings (whatToShow) are as follows:

<img src="./images/fig_10_07.png" width=1000>
Figure 10.7: Valid data types to return for historical data

요청을 보낸 후, 메소드는 데이터를 검색하고 `historical_data` 딕셔너리에 `request_id`를 키로 사용하여 저장할 시간을 확보하기 위해 실행을 5초 동안 일시 중지합니다.

> **⚠️ 중요한 참고사항 (IMPORTANT NOTE)**  
>
> 5초의 지연은 임의적이며 모든 데이터가 수신되었음을 보장하지 않습니다. 필요에 따라 이 지연 시간을 연장하거나 `IBWrapper` 클래스의 `historicData` 메소드에서 데이터가 존재하는지 확인하는 더 정교한 방법을 구현할 수 있습니다.

그 후, 메소드는 요청된 데이터가 일중(intraday) 데이터인지 여부에 따라 들어오는 시간 문자열을 해석하는 방법을 결정합니다.

다음으로, 메소드는 과거 데이터를 검색하고 해당 데이터를 사용하여 pandas DataFrame을 생성합니다. DataFrame의 열(columns)은 `TRADE_BAR_PROPERTIES`에 정의된 열 이름으로 구성됩니다. DataFrame의 인덱스는 `time` 열로 설정되며, 해당 열은 포맷팅된 후 원본 `time` 열은 삭제됩니다. 추가로, DataFrame에는 계약의 심볼(symbol)을 포함하는 새 `symbol` 열이 추가되며, `request_id`는 DataFrame의 속성으로 저장됩니다.

**과거 데이터 수신하기 (Receiving historic data)**

- `historicalData` 메소드는 콜백(callback)으로, `reqHistoricalData` 메소드가 IB에 과거 시장 데이터를 요청하면 IB는 해당 데이터를 개별 바(bar) 단위로 다시 보내며, 각 바가 `historicalData` 메소드를 트리거합니다. 이 메소드는 각 데이터 조각을 처리하고 저장합니다.

- `historicalData`는 두 가지 매개변수를 받습니다:  
  - **`request_id`**: 특정 데이터 요청과 연결된 고유 식별자로, 이 메소드는 이를 통해 다른 요청의 데이터와 구분할 수 있습니다.  
  - **`bar`**: 해당 기간에 대한 과거 시장 데이터를 포함하는 객체로, 다음과 같이 구성됩니다.  
    - `bar.date`: 날짜  
    - `bar.open`: 시가  
    - `bar.high`: 고가  
    - `bar.low`: 저가  
    - `bar.close`: 종가  
    - `bar.volume`: 거래량  

- 메소드는 먼저 `request_id`가 이미 `historical_data` 딕셔너리에 키로 존재하는지 확인합니다. 해당 키가 없으면 해당 키에 대한 빈 리스트를 초기화합니다. 이 단계는 각 고유 요청의 과거 데이터를 저장할 구조를 보장합니다.

- 마지막으로, 메소드는 `bar_data` 튜플을 `historical_data` 딕셔너리의 `request_id`에 연결된 리스트에 추가합니다. 이 작업은 시간이 지남에 따라 각 요청의 과거 데이터 포인트를 축적하도록 합니다.

- **`get_historical_data_for_many`** 메소드는 제공된 각 계약(contract)을 순차적으로 순회하며, 과거 데이터를 요청하고, 그 결과 DataFrame을 결합합니다. 메소드는 피벗(pivot)된 DataFrame을 반환하며, 여기서 `col_to_use`에 지정된 데이터는 행(rows)에, 요청된 각 계약은 열(columns)에 배치됩니다.

---

### **더 있습니다... (There's more...)**

이제 과거 시장 데이터를 요청하고 수신하는 코드를 작성했습니다. `app.py` 파일을 열고 계약 정의 아래에 다음 코드 블록을 추가하세요.

```python
data = app.get_historical_data(
    request_id=99,
    contract=aapl,
    duration='2 D',
    bar_size='30 secs'
)
```

변경사항을 적용한 후 `app.py` 파일의 코드는 다음과 같습니다.

```python
import threading
import time
from wrapper import IBWrapper
from client import IBClient
from contract import stock, future, option

class IBApp(IBWrapper, IBClient):
    def __init__(self, ip, port, client_id):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)
        self.connect(ip, port, client_id)
        thread = threading.Thread(target=self.run, daemon=True)
        thread.start()
        time.sleep(2)

if __name__ == "__main__":
    app = IBApp("127.0.0.1", 7497, client_id=10)
    aapl = stock("AAPL", "SMART", "USD")
    gbl = future("GBL", "EUREX", "202403")
    pltr = option("PLTR", "BOX", "20240315", 20, "C")
    data = app.get_historical_data(
        request_id=99,
        contract=aapl,
        duration='2 D',
        bar_size='30 secs'
    )
    time.sleep(30)
    app.disconnect()
```

이 코드를 실행한 후, 변수 `data`에는 요청된 과거 시장 데이터를 포함하는 pandas DataFrame이 저장됩니다.

<img src="./images/fig_10_08.png" width=800>

Figure 10.8: A pandas DataFrame containing requested historic market data


> **💡 팁 (TIP)**  
>
> PyCharm 또는 VSCode와 같은 **인터랙티브 개발 환경(IDE)**을 사용하는 경우, 개발 중에 디버그 모드에서 코드를 실행할 수 있습니다. 디버그 모드를 사용하면 코드 실행을 일시 중지하여 변수를 검사할 수 있습니다.

---

### **참고사항 (See also)**

과거 시장 데이터에 대해 더 알고 싶다면 다음 URL의 문서를 참조하세요: [https://interactivebrokers.github.io/tws-api/historical_data.html](https://interactivebrokers.github.io/tws-api/historical_data.html)  

특정 주제에 대한 문서는 다음과 같습니다:
- **과거 시장 데이터 요청하기:** [https://interactivebrokers.github.io/tws-api/historical_bars.html#hd_request](https://interactivebrokers.github.io/tws-api/historical_bars.html#hd_request)  
- **과거 시장 데이터 수신하기:** [https://interactivebrokers.github.io/tws-api/historical_bars.html#hd_receive](https://interactivebrokers.github.io/tws-api/historical_bars.html#hd_receive)  
- **기간, 바 크기, 데이터 유형 및 금융 상품별 사용 가능한 데이터:** [https://interactivebrokers.github.io/tws-api/historical_bars.html#hd_duration](https://interactivebrokers.github.io/tws-api/historical_bars.html#hd_duration)  

---

## **시장 데이터 스냅샷 가져오기**

이전 레시피에서는 과거 시장 데이터를 가져오는 방법을 배웠습니다. 일부 상황에서는 현재 시장 가격이 필요할 수 있습니다.  
**Chapter 12, Deploy Strategies to a Live Environment**에서는 포트폴리오의 특정 가치 또는 비율 할당을 목표로 하는 방법을 만들기 위해 현재 시장 가격을 사용할 것입니다.

IB API는 시장 데이터의 특정 범주를 나타내는 틱(tick) 유형을 사용합니다. 예: 마지막 거래 가격, 거래량, 매도호가 및 매수호가. 이러한 틱 유형을 통해 실시간 가격 정보를 확인할 수 있으므로, 정보에 입각한 거래 결정을 내리는 데 중요합니다. 이번 레시피에서는 실시간 시장 데이터를 가져오는 방법을 보여줍니다.

---

### **준비하기...**

`trading-app` 디렉토리에 `app.py`, `client.py`, 및 `wrapper.py` 파일을 생성했다고 가정합니다. 그렇지 않다면 지금 생성하세요.

---

### **어떻게 하는지...**

우리는 `app.py`, `client.py`, 및 `wrapper.py` 파일을 업데이트하여 계약의 마지막 가격을 요청할 것입니다.

1. **`client.py`를 열고, `get_historical_data_for_many` 메소드 아래에 `IBClient` 클래스에 다음 메소드를 추가하세요.**

```python
def get_market_data(self, request_id, contract, tick_type=4):
    self.reqMktData(
        reqId=request_id,
        contract=contract,
        genericTickList="",
        snapshot=True,
        regulatorySnapshot=False,
        mktDataOptions=[]
    )
    time.sleep(5)
    return self.market_data[request_id].get(tick_type)
```

2. **`wrapper.py`를 열고, `historicalData` 메소드 아래에 `IBWrapper` 클래스에 다음 메소드를 추가하세요.**

```python
def tickPrice(self, request_id, tick_type, price, attrib):
    if request_id not in self.market_data.keys():
        self.market_data[request_id] = {}
    self.market_data[request_id][tick_type] = float(price)
```

3. **`__init__` 메소드에 `self.historical_data = {}` 아래에 다음 인스턴스 변수를 추가하세요.**

```python
self.market_data = {}
```

---

### **작동 방식...**

`get_market_data` 메서드는 IB API에서 특정 틱 유형을 가져옵니다. 이는 `request_id`로 식별되는 금융 계약에 대한 시장 데이터 요청을 시작하며, `genericTickList` 인수에 빈 문자열을 사용하여 사용 가능한 모든 틱을 요청합니다. 해당 함수는 연속 스트림이 아닌 단일 데이터 스냅샷을 선택하며, 데이터 수신 및 처리를 위해 5초 동안 멈춥니다. 일시 정지 후에는 요청 ID를 해제하여 요청을 취소합니다. 마지막으로, 함수는 요청 ID와 틱 유형을 키로 사용하여 사전(Dictionary)에서 특정 시장 데이터를 검색하여 반환합니다. 기본값으로 틱 유형 4(마지막 거래 가격)를 사용합니다.

`tickPrice` 콜백에서는 먼저 `request_id`가 `market_data` 사전에 존재하는지 확인하며, 존재하지 않으면 해당 ID에 대한 빈 사전을 초기화합니다. 그런 다음, 사전을 업데이트하여 해당 `tick_type` 키에 수신된 `price`를 float 형식으로 변환하여 설정합니다.

`tickPrice` 메서드는 일반적으로 7가지 다른 틱 유형을 검색합니다.

<img src="./images/fig_10_09.png" width=800>

Figure 10.9: Different tick types returned by the IB API

---

### **더보기...**

AAPL의 마지막 종가를 가져오려면, AAPL 계약을 정의한 후 `app.py`에 다음 코드를 추가하세요:

```python
data = app.get_market_data(
    request_id=99,
    contract=aapl
)
```

`data` 변수에는 AAPL의 마지막 거래 가격을 나타내는 float 값이 들어 있습니다.

---

### 참고 자료

IB API를 사용하여 시장 데이터를 요청하고 수신하는 방법에 대해 더 알아보려면 다음 리소스를 참조하세요:

- 다양한 틱 유형 목록: [https://interactivebrokers.github.io/tws-api/tick_types.html](https://interactivebrokers.github.io/tws-api/tick_types.html)
- 시장 데이터 요청: [https://interactivebrokers.github.io/tws-api/md_request.html](https://interactivebrokers.github.io/tws-api/md_request.html)
- 시장 데이터 수신: [https://interactivebrokers.github.io/tws-api/md_receive.html](https://interactivebrokers.github.io/tws-api/md_receive.html)

---

## 실시간 시장 데이터 스트리밍

틱 단위의 데이터를 요청하는 것은 실시간으로 세밀한 시장 데이터를 획득하는 방식을 포함합니다. 이 과정은 `reqTickByTickData` 메서드를 호출하여 시작됩니다. 여기서 고유한 요청 식별자, 금융 상품의 계약 세부 정보 및 관심 있는 틱 데이터 유형(예: `BidAsk`)을 지정합니다. 이 요청이 이루어지면 IB API는 각 개별 시장 **틱**(시장 데이터의 단일 변경 또는 업데이트)의 데이터를 전송하기 시작합니다.

대량의 과거 데이터 검색과 달리 이 방법은 시장 이벤트가 발생함과 동시에 데이터를 거의 즉각적으로 제공합니다. 이를 통해 거의 실시간 수준의 알고리즘 트레이딩 애플리케이션을 구축할 수 있습니다. 수신된 데이터는 각 틱마다 트리거되는 콜백 함수를 통해 처리되며, 가격, 크기, 틱 발생 시간과 같은 세부 정보를 포착합니다. 이 가이드를 완료하면 IB API에서 거의 실시간 시장 데이터를 스트리밍할 수 있게 됩니다.

---

### **준비하기...**


다음 파일들이 `trading-app` 디렉토리에 생성되었다고 가정합니다: `app.py`, `client.py`, `wrapper.py`. 아직 생성하지 않았다면 지금 생성하세요.

---

### **어떻게 하는지...**

`app.py`, `client.py`, 그리고 `wrapper.py` 파일을 업데이트하여 과거 시장 데이터를 요청합니다.

1. `client.py` 파일을 열고 파일 상단에 다음 import 구문을 추가하세요:

```python
from dataclasses import import dataclass, field
```

2. 그런 다음, `TRADE_BAR_PROPERTIES` 상수 아래에 각 가격 틱을 나타내는 `dataclass`를 추가하세요:

```python
@dataclass
class Tick:
    time: int
    bid_price: float
    ask_price: float
    bid_size: float
    ask_size: float
    timestamp_: pd.Timestamp = field(init=False)

    def __post_init__(self):
        self.timestamp_ = pd.to_datetime(self.time, unit="s")
        self.bid_price = float(self.bid_price)
        self.ask_price = float(self.ask_price)
        self.bid_size = int(self.bid_size)
        self.ask_size = int(self.ask_size)
```

3. `client.py` 파일의 `IBClient` 클래스 안에 스트리밍 데이터를 시작 및 중지하는 함수를 추가하세요:

```python
def get_streaming_data(self, request_id, contract):
    self.reqTickByTickData(
        reqId=request_id,
        contract=contract,
        tickType="BidAsk",
        numberOfTicks=0,
        ignoreSize=True
    )
    time.sleep(10)
    while True:
        if self.stream_event.is_set():
            yield Tick(
                *self.streaming_data[request_id]
            )
            self.stream_event.clear()

def stop_streaming_data(self, request_id):
    self.cancelTickByTickData(reqId=request_id)
```

다음은 수정된 `client.py`의 전체 코드입니다:

```python
import time
import pandas as pd
from dataclasses import dataclass, field
from ibapi.client import EClient

TRADE_BAR_PROPERTIES = ["time", "open", "high", "low", "close", "volume"]

@dataclass
class Tick:
    time: int
    bid_price: float
    ask_price: float
    bid_size: float
    ask_size: float
    timestamp_: pd.Timestamp = field(init=False)

    def __post_init__(self):
        self.timestamp_ = pd.to_datetime(self.time, unit="s")
        self.bid_price = float(self.bid_price)
        self.ask_price = float(self.ask_price)
        self.bid_size = int(self.bid_size)
        self.ask_size = int(self.ask_size)

class IBClient(EClient):
    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)

    def get_streaming_data(self, request_id, contract):
        self.reqTickByTickData(
            reqId=request_id,
            contract=contract,
            tickType="BidAsk",
            numberOfTicks=0,
            ignoreSize=True
        )
        time.sleep(10)
        while True:
            if self.stream_event.is_set():
                yield Tick(
                    *self.streaming_data[request_id]
                )
                self.stream_event.clear()

    def stop_streaming_data(self, request_id):
        self.cancelTickByTickData(reqId=request_id)
```

4. `wrapper.py` 파일의 상단에 다음 import 구문을 추가하세요:

```python
import threading
```

5. `IBWrapper` 클래스의 `__init__` 메서드에 다음 코드를 추가하세요:

```python
self.streaming_data = {}
self.stream_event = threading.Event()
```

6. 다음 메서드를 `IBWrapper` 클래스에 추가하세요:

```python
def tickByTickBidAsk(
    self,
    request_id,
    time,
    bid_price,
    ask_price,
    bid_size,
    ask_size,
    tick_attrib_last
):
    tick_data = (
        time,
        bid_price,
        ask_price,
        bid_size,
        ask_size,
    )
    self.streaming_data[request_id] = tick_data
    self.stream_event.set()
```

다음은 수정된 `wrapper.py`의 전체 코드입니다:

```python
import threading
from ibapi.wrapper import EWrapper

class IBWrapper(EWrapper):
    def __init__(self):
        EWrapper.__init__(self)
        self.nextValidOrderId = None
        self.historical_data = {}
        self.streaming_data = {}
        self.stream_event = threading.Event()

    <snip>

    def tickByTickBidAsk(
        self,
        request_id,
        time,
        bid_price,
        ask_price,
        bid_size,
        ask_size,
        tick_attrib_last
    ):
        tick_data = (
            time,
            bid_price,
            ask_price,
            bid_size,
            ask_size,
        )
        self.streaming_data[request_id] = tick_data
        self.stream_event.set()
```

---

### **작동 방식...**

IB API를 통해 스트리밍 데이터를 가져오려면 두 단계가 필요합니다: 먼저 요청하고, 그 후 콜백을 통해 처리합니다. 우리가 스트리밍하는 틱 단위의 데이터는 TWS Time and Sales 창에 표시되는 데이터에 해당합니다.

> **중요 참고사항**  
>
> Time and Sales는 특정 금융 상품에 대해 실행된 거래에 대한 실시간 데이터 피드를 의미합니다. 이 피드에는 각 거래가 실행된 정확한 시간, 거래가 체결된 가격 및 거래의 크기가 포함됩니다. Time and Sales 데이터는 주문 장의 동적 특성을 실시간으로 분석하려는 트레이더에게 유용합니다.

**스트리밍 데이터 요청하기**

먼저, Python의 `dataclass`인 `Tick`을 정의합니다.  
Python의 `dataclass`는 클래스에 `__init__`, `__repr__`, `__eq__`와 같은 특수 메서드를 자동으로 생성하는 데코레이터로, 주로 데이터를 저장하는 데 사용되며 보일러플레이트 코드를 줄입니다.

- `Tick dataclass`는 시간, 매수 및 매도 가격, 크기 등의 속성을 가진 시장 데이터 틱을 나타냅니다.  
- 추가 속성인 `timestamp_`는 pandas의 `Timestamp`로 정의되며, `field(init=False)`를 사용하여 자동 `__init__` 메서드 생성에서 제외됩니다.  
- `dataclass`의 `__post_init__` 메서드는 `time` 속성을 Unix 타임스탬프에서 더 읽기 쉬운 pandas의 `Timestamp` 형식으로 변환합니다.

`get_streaming_data` 메서드는 스트리밍 데이터를 시작하기 위해 요청 ID와 계약이 필요합니다. 이 메서드는 지정된 금융 상품의 실시간 매수 및 매도 틱 데이터를 요청하고 이를 반환합니다. 금융 상품은 `contract` 매개변수로 전달됩니다.

메서드는 먼저 IB API의 `reqTickByTickData` 함수를 호출하여 실시간 틱 단위 데이터를 요청합니다. 이 함수에 전달되는 매개변수는 다음과 같습니다:

- **`reqId`**: 데이터 요청을 위한 고유 식별자입니다.  
- **`contract`**: 데이터가 요청되는 금융 상품을 지정합니다.  
- **`tickType`**: 스트리밍할 데이터 유형을 지정합니다. 사용 가능한 옵션은 `Last`, `AllLast`, `BidAsk`, `MidPoint`입니다.  
- **`numberOfTicks`**: 특정 세트의 과거 틱을 요청하거나, 취소될 때까지 스트리밍합니다.  
- **`ignoreSize`**: 매수 및 매도 크기에 대한 업데이트가 필요한지 여부를 지정합니다.

데이터 요청이 완료되면, 데이터가 수신되고 버퍼링될 시간을 확보하기 위해 10초 동안 지연을 줍니다.  
그 후 메서드는 무한 루프에 들어가, `stream_event` 플래그가 설정되었는지 지속적으로 확인하며, 설정되었을 때 주어진 `request_id`에 대한 최신 데이터로 `Tick` 객체를 반환합니다. 그런 다음 이벤트를 지워 다음 데이터 배치를 받을 준비를 합니다.

> **중요 참고사항**  
>
> Python의 제너레이터(generator)는 데이터를 한 번에 모두 메모리에 저장하지 않고, 필요할 때마다 항목을 생성하는 특수 유형의 함수입니다. 제너레이터 함수가 호출되면 즉시 코드를 실행하지 않고, 반복할 수 있는 제너레이터 객체를 반환합니다. 제너레이터 객체를 반복(iterate)할 때마다 함수는 마지막으로 값을 반환했던 지점에서 실행을 재개하며, 다른 `yield`에 도달하거나 함수의 끝에 도달하면 `StopIteration` 예외를 발생시킵니다.

**스트리밍 데이터 수신하기**

`reqTickByTickData`에서 요청한 `tickType`에 따라, IB는 서로 다른 콜백 함수를 사용합니다. 콜백 함수는 다음과 같습니다:  

<img src="./images/fig_10_10.png" width=800>

Figure 10.10: Callbacks used for each tick type requested

> **중요 참고사항**  
> 우리는 `tickByTickBidAsk`만 구현하여 `BidAsk` 틱 유형에 응답합니다. 각 콜백에서 반환되는 데이터 유형에 대해 더 알고 싶다면 다음 URL을 참조하세요: [https://interactivebrokers.github.io/tws-api/tick_data.html](https://interactivebrokers.github.io/tws-api/tick_data.html)

우리는 IB API에서 제공하는 `tickByTickBidAsk` 메서드를 재정의합니다. 이 콜백 메서드는 각 틱마다 호출됩니다.  
호출 시, 메서드는 먼저 시간, 매수 가격, 매도 가격, 매수 크기, 매도 크기로 구성된 튜플인 `tick_data`를 생성합니다.  
그런 다음, 해당 `request_id`를 키로 사용하여 `streaming_data` 사전을 업데이트하며, `stream_event` 스레딩 이벤트를 설정하여 새로운 데이터가 수신되었고 `get_streaming_data` 메서드에서 처리할 준비가 되었음을 알립니다.

---


### **더보기...**

이제 스트리밍 틱 데이터를 요청하고 수신하기 위한 코드가 준비되었습니다.  
`app.py` 파일을 열고, 계약 정의 아래에 다음 코드 블록을 추가하세요:

```python
eur = future("EUR", "CME", "202312")
for tick in app.get_streaming_data(99, eur):
    print(tick)
```

다음은 수정된 `app.py`의 전체 코드입니다:

```python
import threading
import time
from wrapper import IBWrapper
from client import IBClient
from contract import future

class IBApp(IBWrapper, IBClient):
    def __init__(self, ip, port, client_id):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)
        self.connect(ip, port, client_id)
        
        thread = threading.Thread(target=self.run, daemon=True)
        thread.start()
        
        time.sleep(2)

if __name__ == "__main__":
    app = IBApp("127.0.0.1", 7497, client_id=10)
    eur = future("EUR", "CME", "202312")
    for tick in app.get_streaming_data(99, eur):
        print(tick)
    time.sleep(30)
    app.disconnect()
```

코드를 실행하면 각 틱의 세부 정보가 포함된 `Tick` 객체 목록이 콘솔에 표시됩니다.

<img src="./images/fig_10_11.png" width=800>

Figure 10.11: Streaming tick data as Tick objects

---

### **참고 자료 (See also)**

스트리밍 데이터의 사용 가능한 유형에 대해 더 알고 싶다면 다음 URL을 참조하세요:  
- [https://interactivebrokers.github.io/tws-api/market_data.html](https://interactivebrokers.github.io/tws-api/market_data.html)  
- 특정 사항은 아래 URL에서 확인하세요:  
  - **Level 1 (top of book)** 스트리밍 및 스냅샷 데이터 (본 레시피에서는 다루지 않음): [https://interactivebrokers.github.io/tws-api/md_request.html](https://interactivebrokers.github.io/tws-api/md_request.html)  
  - **스트리밍 틱 데이터** (본 레시피에서 다룸): [https://interactivebrokers.github.io/tws-api/tick_data.html](https://interactivebrokers.github.io/tws-api/tick_data.html)

---

## **실시간 틱 데이터를 로컬 SQL 데이터베이스에 저장하기**

금융 시장 데이터를 저장하는 방법에 대해 **Chapter 4**인 *Store Financial Market Data on Your Computer*에서 설명했습니다.  
이 장에서는 다양한 유형의 시장 데이터를 무료로 다운로드하고 다양한 형식으로 저장하는 방법을 배웠습니다.  
**Chapter 13**, *Advanced Recipes for Market Data and Strategy Management*에서는 최첨단 **ArcCDB** 라이브러리를 사용하여 페타바이트급 시장 데이터를 저장하는 방법을 다룹니다.  
이번에는 이 장과 **Chapter 4**의 레시피를 확장하여 스트리밍 틱 데이터를 로컬 **SQLite** 데이터베이스에 저장하는 방법을 설명합니다.

---

### **준비하기...**


- 우리는 사용자가 **Chapter 4**의 *Storing Data On-Disk with SQLite*를 읽었고, `trading-app` 디렉토리에 `app.py`, `client.py`, `wrapper.py`, `utils.py` 파일을 생성했다고 가정합니다.  
- 아직 생성하지 않았다면 지금 생성하세요.

현재 `client.py`에 있는 일부 코드를 `utils.py` 파일로 이동하겠습니다.  
아래 코드를 `client.py`에서 복사하여 `utils.py`에 붙여넣으세요:

```python
import pandas as pd
from dataclasses import dataclass, field

TRADE_BAR_PROPERTIES = ["time", "open", "high", "low", "close", "volume"]

@dataclass
class Tick:
    time: int
    bid_price: float
    ask_price: float
    bid_size: float
    ask_size: float
    timestamp_: pd.Timestamp = field(init=False)

    def __post_init__(self):
        self.timestamp_ = pd.to_datetime(self.time, unit="s")
        self.bid_price = float(self.bid_price)
        self.ask_price = float(self.ask_price)
        self.bid_size = int(self.bid_size)
        self.ask_size = int(self.ask_size)
```

이제 `client.py`에서 해당 코드를 제거하고, `utils.py`에서 가져오도록 수정하세요.  
변경사항 적용 결과는 다음과 같습니다:

```python
import time
import pandas as pd
from utils import Tick, TRADE_BAR_PROPERTIES
from ibapi.client import EClient

class IBClient(EClient):
    <snip>
```

---


### **어떻게 하는지...**

`IBApp` 클래스에 메서드를 추가하여 **SQLite** 데이터베이스를 생성하고, 데이터베이스에 연결하며, 틱 데이터를 테이블에 삽입하도록 합니다.

**1.** `app.py` 파일의 `IBApp` 클래스에 `connection` 메서드를 추가하고 `@property` 데코레이터로 장식하세요:

```python
@property
def connection(self):
    return sqlite3.connect("tick_data.sqlite", isolation_level=None)
```

**2.** `IBApp` 클래스에 `create_table` 메서드를 추가하세요:

```python
def create_table(self):
    cursor = self.connection.cursor()
    cursor.execute(
        """CREATE TABLE IF NOT EXISTS bid_ask_data (
            timestamp datetime,
            symbol string,
            bid_price real,
            ask_price real,
            bid_size integer,
            ask_size integer)"""
    )
```

**3.** `IBApp` 클래스에 `stream_to_sqlite` 메서드를 추가하세요. 걱정하지 마세요. 다음에 자세히 설명하겠습니다:

```python
def stream_to_sqlite(self, request_id, contract, run_for_in_seconds=23400):
    cursor = self.connection.cursor()
    end_time = time.time() + run_for_in_seconds + 10
    for tick in app.get_streaming_data(request_id, contract):
        query = """INSERT INTO bid_ask_data (
            timestamp, symbol, bid_price,
            ask_price, bid_size, ask_size)
            VALUES (?, ?, ?, ?, ?, ?)"""
        values = (
            tick.timestamp_.strftime("%Y-%m-%d %H:%M:%S"),
            contract.symbol,
            tick.bid_price,
            tick.ask_price,
            tick.bid_size,
            tick.ask_size
        )
        cursor.execute(query, values)
        if time.time() >= end_time:
            break
    self.stop_streaming_data(request_id)
```

**4.**. `IBApp` 클래스의 `__init__` 메서드에서 `IBWrapper` 및 `IBClient` 호출 아래에 다음 코드를 추가하세요:  

```python
self.create_table()
```

다음은 수정된 `IBApp` 클래스의 전체 코드입니다:

```python
class IBApp(IBWrapper, IBClient):
    def __init__(self, self, ip, port, client_id):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)
        self.create_table()
        self.connect(ip, port, client_id)
        
        thread = threading.Thread(target=self.run, daemon=True)
        thread.start()
        time.sleep(2)

    @property
    def connection(self):
        return sqlite3.connect("tick_data.sqlite", isolation_level=None)

    def create_table(self):
        cursor = self.connection.cursor()
        cursor.execute(
            """CREATE TABLE IF NOT EXISTS bid_ask_data (
                timestamp datetime,
                symbol string,
                bid_price real,
                ask_price real,
                bid_size integer,
                ask_size integer)"""
        )

    def stream_to_sqlite(self, request_id, contract, run_for_in_seconds=23400):
        cursor = self.connection.cursor()
        end_time = time.time() + run_for_in_seconds + 10
        for tick in app.get_streaming_data(request_id, contract):
            query = """INSERT INTO bid_ask_data (
                timestamp, symbol, bid_price,
                ask_price, bid_size, ask_size) VALUES (
                ?, ?, ?, ?, ?, ?)"""
            values = (
                tick.timestamp_.strftime("%Y-%m-%d %H:%M:%S"),
                contract.symbol,
                tick.bid_price,
                tick.ask_price,
                tick.bid_size,
                tick.ask_size
            )
            cursor.execute(query, values)
            if time.time() >= end_time:
                break
        self.stop_streaming_data(request_id)
```

---

### **작동 원리...**

우선, `connection`이라는 클래스 속성을 생성하여 SQLite 데이터베이스에 대한 연결을 설정하고 반환합니다. `@property` 데코레이터를 사용하여 `connection` 메소드를 메소드처럼 호출하지 않고 속성처럼 액세스할 수 있습니다.

`connection` 속성이 액세스되면 `tick_data.sqlite`라는 이름의 SQLite 데이터베이스 파일에 대한 연결이 생성됩니다. `isolation_level=None` 매개변수는 트랜잭션 격리 수준을 `None`으로 설정하며, 이는 **autocommit** 모드가 활성화됨을 의미합니다. **autocommit** 모드에서는 각 데이터베이스 작업 후 명시적으로 `commit`을 호출하지 않아도 변경 사항이 즉시 커밋됩니다.

`create_table` 메소드는 `bid_ask_data`라는 이름의 데이터베이스 테이블을 생성합니다. 이 메소드가 호출되면, 먼저 데이터베이스 커서를 생성하여 데이터베이스 명령어를 실행합니다. 그런 다음, `execute` 메소드를 사용하여 테이블이 존재하지 않으면 이를 생성하는 **CREATE** SQL 명령어를 실행합니다. `IBApp`가 초기화되면 `self.create_table`을 호출하여 테이블이 존재하지 않을 경우 생성합니다.

`stream_to_sqlite` 메소드는 지정된 기간 동안 SQLite 데이터베이스에 틱 데이터를 저장합니다. 먼저, 생성된 SQLite 연결로부터 데이터베이스 커서를 생성합니다. 그런 다음, 현재 시간에 `run_for_in_seconds`로 지정된 기간(기본값은 23,400초 또는 6.5시간)을 더하여 `end_time` 값을 계산합니다. 여기에 틱 스트림이 시작될 때까지의 대기 시간을 보완하기 위해 추가로 10초의 버퍼가 더해집니다. 이후 루프에 들어가 `get_streaming_data` 메소드로부터 틱 객체(`Tick`)를 가져오는데, 이는 주어진 `request_id` 및 `contract`의 시장 데이터를 포함합니다. 각 틱이 수신될 때마다 해당 데이터를 `bid_ask_data` 테이블에 추가하는 SQL **INSERT** 쿼리를 생성하여 데이터를 추가합니다. 이 과정에서 타임스탬프를 포맷하고 계약의 심볼(symbol), 매수호가(bid price), 매도호가(ask price), 매수 수량(bid size), 및 매도 수량(ask size)과 같은 관련 데이터를 포함합니다. 루프는 현재 시간이 계산된 `end_time`을 초과할 때까지 계속되며, 그 시점에서 중단되어 데이터 스트리밍이 지정된 기간 내로 제한됩니다.

---

### **또 다른 점은...**

이제 스트리밍 틱 데이터를 저장할 수 있도록 코드가 구성되었습니다. 계약 정의 아래에 다음 코드 블록을 추가하십시오:

```python
es = future("ES", "CME", "202312")
app.stream_to_sqlite(99, es, run_for_in_seconds=30)
```

변경된 코드의 최종 결과는 `app.py` 파일의 다음 코드와 같습니다:

```python
<snip>
if __name__ == "__main__":
    app = IBApp("127.0.0.1", 7497, client_id=10)
    es = future("ES", "CME", "202312")
    app.stream_to_sqlite(99, es, run_for_in_seconds=30)
    app.disconnect()
```

SQLite 데이터베이스에 저장된 데이터를 시각적으로 확인하려면, **DB Browser for SQLite**를 사용할 수 있습니다. 이는 SQLite 데이터베이스에 저장된 데이터를 그래픽 인터페이스를 통해 검사할 수 있는 무료 도구입니다. 다운로드 링크: [https://sqlitebrowser.org](https://sqlitebrowser.org). 설치 후 `tick_data.sqlite` 파일을 열고 **Browse Data** 탭으로 이동하면 저장된 데이터를 확인할 수 있습니다.

<img src="./images/fig_10_12.png" width=800>

Figure 10.12: Browsing bid ask tick data in our SQLite database 

### ​See also

The SQLite documentation​ was covered in Chapter 4, Store Financial Market Data on Your Computer, but if you’re interested in more​ details about the Connection and Cursor objects, you can check the following URLs: 
- Documentation for the SQLite connect method: https://docs.python.org/3/library/sqlite3.html#sqlite3.connect 
- Documentation for the SQLite Cursor object: https://docs.python.org/3/library/sqlite3.html#cursor-objects

---